<a href="https://colab.research.google.com/github/chaotingchong-crypto/programming-language/blob/main/Hw6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# 🔔 課表查詢 + 上課提醒 + AI 行前準備（Gradio + Colab 版）
# 使用方式：
# 1. 在 Colab 新增程式碼格，貼上這整段程式
# 2. 先執行一次，授權 Google 帳號
# 3. 介面會出現 Gradio 網頁，可直接操作

!pip install --quiet gradio gspread pandas gspread_dataframe google-auth

import pandas as pd
from datetime import datetime, timedelta, time
import gradio as gr
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import get_as_dataframe

# === 1️⃣ Google 認證 ===
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# === 2️⃣ 讀取你的 Google Sheet ===
sheet_url = "https://docs.google.com/spreadsheets/d/1nU6gZU2AFrGZk1BkABC60BEFWTcR2akzIoLh-4mRSQ4/edit?hl=zh-tw&gid=0#gid=0"  # <-- 這裡換成你的 Sheet 網址
workbook = gc.open_by_url(sheet_url)
worksheet = workbook.sheet1  # 或改成 workbook.worksheet("分頁名稱")

# 讀取成 DataFrame，只保留必要欄位
df = get_as_dataframe(worksheet)
df = df[['課程中文名稱', '上課教室', '星期', '節次', '上課時間']]

# === 3️⃣ 輔助函數：解析開始時間 ===
def parse_start_time(time_str):
    """
    將『14:20 - 15:10』這種字串，取出開始時間 14:20 轉成 datetime.time
    """
    if not isinstance(time_str, str):
        return None
    try:
        start_part = time_str.split("-")[0].strip()
        hour, minute = start_part.split(":")
        return time(int(hour), int(minute))
    except:
        return None

df["開始時間_obj"] = df["上課時間"].apply(parse_start_time)

# === 4️⃣ 星期對照表 ===
weekday_map_num_to_tw = {
    0: "星期一",
    1: "星期二",
    2: "星期三",
    3: "星期四",
    4: "星期五",
    5: "星期六",
    6: "星期日",
}

# === 5️⃣ 查詢某一天課表 ===
def query_by_weekday_gr(weekday_tw):
    if not weekday_tw:
        return "⚠️ 請先選擇星期"
    mask = df["星期"] == weekday_tw
    day_df = df[mask].sort_values(by="節次")
    if day_df.empty:
        return f"📅 {weekday_tw} 沒有課喔～"

    lines = [f"📅 {weekday_tw} 課表：\n"]
    for _, row in day_df.iterrows():
        lines.append(
            f"節次：{row['節次']}\n"
            f"時間：{row['上課時間']}\n"
            f"課程：{row['課程中文名稱']}\n"
            f"地點：{row['上課教室']}\n"
            "--------------------------"
        )
    return "\n".join(lines)

# === 6️⃣ 今日即將上課提醒 ===
def upcoming_classes_gr(minutes_ahead):
    try:
        minutes_ahead = int(minutes_ahead)
    except:
        return "⚠️ 分鐘數需為正整數"
    if minutes_ahead <= 0:
        return "⚠️ 分鐘數需大於 0"

    now = datetime.now()
    today_tw = weekday_map_num_to_tw.get(now.weekday(), "未知星期")
    now_time = now.time()
    later_time = (now + timedelta(minutes=minutes_ahead)).time()

    today_df = df[df["星期"] == today_tw].dropna(subset=["開始時間_obj"])
    upcoming = today_df[(today_df["開始時間_obj"] >= now_time) &
                        (today_df["開始時間_obj"] <= later_time)].sort_values(by="開始時間_obj")

    header_lines = [
        f"⏰ 現在時間：{now.strftime('%Y-%m-%d %H:%M')}",
        f"📅 今天是：{today_tw}\n"
    ]

    if upcoming.empty:
        header_lines.append(f"未來 {minutes_ahead} 分鐘內沒有新的課要上～")
        return "\n".join(header_lines)

    header_lines.append(f"未來 {minutes_ahead} 分鐘內即將開始的課程：\n")
    for _, row in upcoming.iterrows():
        header_lines.append(
            f"時間：{row['上課時間']}\n"
            f"課程：{row['課程中文名稱']}\n"
            f"地點：{row['上課教室']}\n"
            "--------------------------"
        )
    return "\n".join(header_lines)

# === 7️⃣ AI 行前準備建議（rule-based，小 AI，不需 API） ===
def ai_prep_tips(weekday_tw):
    if not weekday_tw:
        return "⚠️ 請先選擇星期"

    day_df = df[df["星期"] == weekday_tw].sort_values(by="節次")
    if day_df.empty:
        return f"📅 {weekday_tw} 沒有課，不用特別準備～好好休息 😆"

    lines = [f"🤖 {weekday_tw} 行前準備建議\n", "📚 今天你有這些課："]
    for _, row in day_df.iterrows():
        lines.append(f"- {row['上課時間']}《{row['課程中文名稱']}》 @ {row['上課教室']}")
    lines.append("\n🧠 全日通用建議：")
    lines.append("• 學生證、錢包、悠遊卡/一卡通")
    lines.append("• 筆記本、原子筆/自動鉛筆、橡皮擦")
    lines.append("• 手機充電、行動電源")
    lines.append("• 水壺、雨具或外套")
    lines.append("\n🎯 依課程類型提醒：")
    for _, row in day_df.iterrows():
        cname = str(row["課程中文名稱"])
        ctime = row["上課時間"]
        room = row["上課教室"]
        tips = []

        if any(k in cname for k in ["體育","運動","Sport","sports"]):
            tips.append("運動服、運動鞋、毛巾、水壺")
        elif any(k in cname for k in ["實驗","實習","實作","Lab","lab"]):
            tips.append("實驗衣、講義、USB 或雲端備份資料")
        elif any(k in cname for k in ["英文","English","英語"]):
            tips.append("英文課本、單字本、有線耳機")
        elif any(k in cname for k in ["資訊","程式","Programming","電腦"]):
            tips.append("筆電、充電器、隨身碟，確認帳號可登入")
        else:
            tips.append("課本、講義、筆記本，提前整理心情")

        lines.append(f"《{cname}》 ({ctime} @ {room})")
        for t in tips:
            lines.append(f"• 建議準備：{t}")
        lines.append("")

    lines.append("✅ 前一天晚上先把包包收好，早上出門檢查一次即可")
    return "\n".join(lines)

# === 8️⃣ Gradio 介面 ===
with gr.Blocks() as demo:
    gr.Markdown("# 🗓 課表查詢 & 上課提醒 & 🤖 AI 行前準備小助理")

    with gr.Tab("📅 依星期查詢課表"):
        weekday_input = gr.Dropdown(
            choices=["星期一","星期二","星期三","星期四","星期五","星期六","星期日"],
            label="選擇星期",
            value="星期一"
        )
        btn_query = gr.Button("查詢課表")
        output_day = gr.Textbox(label="查詢結果", lines=15, interactive=False)
        btn_query.click(fn=query_by_weekday_gr, inputs=weekday_input, outputs=output_day)

    with gr.Tab("⏰ 今日上課提醒"):
        minutes_input = gr.Slider(minimum=10, maximum=180, step=10, value=60,
                                  label="未來幾分鐘內的課程？")
        btn_upcoming = gr.Button("查詢即將開始的課程")
        output_upcoming = gr.Textbox(label="提醒結果", lines=15, interactive=False)
        btn_upcoming.click(fn=upcoming_classes_gr, inputs=minutes_input, outputs=output_upcoming)

    with gr.Tab("🤖 AI 行前準備"):
        weekday_ai = gr.Dropdown(
            choices=["星期一","星期二","星期三","星期四","星期五","星期六","星期日"],
            label="選擇要準備的星期",
            value="星期一"
        )
        btn_ai = gr.Button("產生行前重點")
        output_ai = gr.Textbox(label="AI 行前準備建議", lines=20, interactive=False)
        btn_ai.click(fn=ai_prep_tips, inputs=weekday_ai, outputs=output_ai)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://53f5fefe9d216d8a54.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
